## Deep Learning Coding Project 2: Image Classification

Before we start, please put your **Chinese** name and student ID in following format:

Name, 0000000000 // e.g.) 傅炜, 2021123123

刘涵祚,2024011303

## Introduction

We will use Python 3, [NumPy](https://numpy.org/), and [PyTorch](https://pytorch.org/) for this coding project. The example code has been tested under the latest stable release version.

### Task

In this notebook, you need to train a model to classify images. Given an image, you need to distinguish its category,
e.g., whether it is a horse or an automobile. There are total 10 classes:
airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. We
release 40,000 images for training, 10,000 images for validation. Each image has
a shape of (3, 128, 128). We will evaluate your model in 10,000 images on the test set.

Download the dataset from [here](https://cloud.tsinghua.edu.cn/d/00e0704738e04d32978b/) and organize them into a folder named "cifar_10_4x".

<!-- Images can be classified as "No Finding" or **one or more types**. In the basic task, given an image, you only need to tell whether the X-ray indicates "Infiltration". In the bonus task, you need to tell whether *each* of the diseases exists.

Images are taken from the [ChestX-ray14 dataset](https://www.kaggle.com/nih-chest-xrays/data) and downsampled to (256, 256). We release 44872 gray scale images for training and validation. We will evaluate your model on 10285 images in the test set. The dataset is available [here](https://cloud.tsinghua.edu.cn/d/16d06a89c5b4459db703/) and organized as follows: `train` directory includes all images for training and validation, and each line of `train.txt` records the labels separated by "|". -->

### Coding

We provide a code template. You can add new cells and modify our example to train your own model. To run this code, you should:

+ implement your model (named `Net`) in `model.py`.
+ implement your training loop in this notebook

Your final submitted model should not be larger than **20M**. **Using any pretrained model is NOT permitted**.
Besides, before you submit your result, **make sure you can test your model using our evaluation cell.** Name your best model "cifar10_4x_best.pth".

### Report & Submission

Your report should include:

1. the details of your model
2. all the hyper-parameters
3. all the tricks or training techniques you use
4. the training curve of your submitted model.

Reporting additional ablation studies and how you improve your model are also encouraged.

You should submit:

+ all codes
+ the model checkpoint (only "cifar10_4x_best.pth")
+ your report (a separate "pdf")

to web learning. We will use the evaluation code in this notebook to evaluate your model on the test set.

### Grading

We will grade this coding project based on the performance of your model (70%) and your report (30%). Regarding the evaluation metric of your model, assume your test accuracy is $X$, then your score is

$\frac{min(X,H)−0.6}{H−0.6}×7$

where $H$ is accuracy of the model trained by TAs and $H=0.9$, i.e., you will get the full score if your test accuracy is above 90%.

**Bonus**: The best submission with the highest testing accuracy will get 1 bonus point for the final course grade.

**Avoid plagiarism! Any student who violates academic integrity will be seriously dealt with and receive an F for the course.**

## Code Template

We have masked the the training loop in this notebook for you to complete. You should also overwrite "model.py" and implement your own model.

In [14]:
# %load_ext autoreload
# %autoreload 2

### Setup Code

If you use Colab in this coding project, please uncomment the code, fill the `GOOGLE_DRIVE_PATH_AFTER_MYDRIVE` and run the following cells to mount your Google drive. Then, the notebook can find the required file. If you run the notebook locally, you can skip the following cells.

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
# import os

# # TODO: Fill in the Google Drive path where you uploaded the assignment
# # Example: If you create a 2022SP folder and put all the files under CP1 folder, then '2022SP/CP1'
# # GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2022SP/CP1'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = None 
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))

In [17]:
# import sys
# sys.path.append(GOOGLE_DRIVE_PATH)

In [18]:
from dataset import CIFAR10_4x
from evaluation import evaluation

from torchvision.transforms.v2 import MixUp
from model_dropout import Net  # this should be implemented by yourself
from torch.utils.data import default_collate

### Enjoy Your Coding Time!

In [19]:
import math
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from PIL import Image


def set_seed(seed):
    seed = int(seed)
    if seed < 0 or seed > (2**32 - 1):
        raise ValueError("Seed must be between 0 and 2**32 - 1")
    else:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
set_seed(16)

In [20]:
data_root_dir = '.'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([125 / 255, 124 / 255, 115 / 255],
                         [60 / 255, 59 / 255, 64 / 255])
    
])


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(128, padding=4),
    transforms.RandomRotation(30),  # 保持旋转角度不变
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),  # 增加颜色抖动的幅度
    transforms.RandomGrayscale(p=0.2),  # 增加随机灰度化的概率
    transforms.RandomAffine(degrees=0, translate=(0.25, 0.25)),  # 增加随机仿射变换的平移幅度
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),  # 增加随机透视变换的失真比例
    # MixUp(alpha=0.2),
    transforms.ToTensor(),
    transforms.Normalize([125 / 255, 124 / 255, 115 / 255],
                         [60 / 255, 59 / 255, 64 / 255]),
])


In [21]:
NUMOFWORKERS = 1
trainset = CIFAR10_4x(root=data_root_dir,
                      split="train", transform=train_transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=NUMOFWORKERS, pin_memory=True)

validset = CIFAR10_4x(root=data_root_dir,
                      split='valid', transform=transform)
validloader = torch.utils.data.DataLoader(
    validset, batch_size=128, shuffle=False, num_workers=NUMOFWORKERS)

Loading CIFAR10_4x at .
Loading file at .\cifar_10_4x\train
Loading CIFAR10_4x at .
Loading file at .\cifar_10_4x\valid


In [22]:
####################################
#                                  #
#                                  #
#                                  #
#                                  #
#        RUN THIS FOR NEW MODEL    #
#                                  #
#                                  #
#                                  #
#                                  #
####################################
net = Net()
print("number of trained parameters: %d" % (
    sum([param.nelement() for param in net.parameters() if param.requires_grad])))
print("number of total parameters: %d" %
      (sum([param.nelement() for param in net.parameters()])))

criterion = nn.CrossEntropyLoss()
train_history = []
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
print(device)
net.to(device)

number of trained parameters: 4740682
number of total parameters: 4740682
cuda


In [23]:
# NO EXECUTION
import os
import torch
import wandb
wandb.init(project='cifar10_4x_omoe')
model_dir = './checkpoint_dropout0.1_'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(os.path.join(model_dir, 'cifar10_4x_0.pth')):
    torch.save(net, os.path.join(model_dir, 'cifar10_4x_0.pth'))

# check the model size
os.system(' '.join(['du', '-h', os.path.join(model_dir, 'cifar10_4x_0.pth')]))

largest = 0
while True:
    if not os.path.exists(os.path.join(model_dir, 'cifar10_4x_%d.pth' % (largest + 1))):
        break
    largest += 1
if os.path.exists(os.path.join(model_dir, f'cifar10_4x_best.pth')):
    net = torch.load(os.path.join(model_dir, f'cifar10_4x_best.pth'), weights_only=False)
else:
    net = torch.load(os.path.join(model_dir, f'cifar10_4x_{largest}.pth'), weights_only=False)
net.to(device)
print(f"Loaded model from cifar10_4x_best.pth")
log_path = os.path.join(model_dir, 'log.json')
#create the file at log_path
log_file = open(log_path, 'w')

Loaded model from cifar10_4x_best.pth


In [24]:
import json
mixup = MixUp(num_classes=10, alpha=0.1)
def train(epochs:int, optimizer, scheduler):
    ##############################################################################
    #                  TODO: You need to complete the code here                  #
    ##############################################################################
    # YOUR CODE HERE
    # print(optimizer.param_groups[0])
    results = []
    best_idx = 0
    best_acc = 0
    from tqdm import tqdm
    from evaluation import evaluation
    best_acc = evaluation(net, validloader, device)
    print(f'best accuracy: {best_acc:.2f}%')
    for epoch in range(epochs):
        if epoch + largest > epochs:
            break
        net.train()
        with tqdm(trainloader) as bar:
            nums = trues = 0
            losses = 0
            for i, (xb, yb) in enumerate(bar): #这里 xb是图片 yb是标签
                xb, yb = xb.to(device), yb.to(device)
                # print(xb.shape, yb.shape)
                xb, yb = mixup(xb, yb)
                # print(xb.shape, yb.shape)
                optimizer.zero_grad()
                final_output, kl_loss_gate, kl_loss_expert_usage = net(xb)
                # print(final_output.shape)
                with torch.no_grad():
                    # _, answer = final_output.max(dim=1)
                    # trues += (answer == yb).sum().item()
                    # nums += len(yb)
                    # now final_output and yb are mixuped logits, shaped as (batch_size, num_classes)
                    trues += (final_output.argmax(dim=1) == yb.argmax(dim=1)).sum().item()
                    nums += yb.shape[0] 
                    # print(f'nums: {nums}, trues: {trues}')
                loss = criterion(final_output, yb) + kl_loss_gate + kl_loss_expert_usage
                losses += loss
                loss.backward()
                optimizer.step()
                if (i % 1 == 0):
                    train_loss = losses / nums
                    bar.set_description(f'Epoch {epoch + largest}/{epochs}, accuracy {100 * trues / nums:.2f}%, train loss {train_loss:.4f}, kl_loss_gate {kl_loss_gate:.4f}, kl_loss_expert_usage {kl_loss_expert_usage:.4f}, learning rate {optimizer.param_groups[0]["lr"]}')
        train_acc = 100 * trues / nums
        print(f'train accuracy: {train_acc:.2f}%')
        net.eval()
        nums = loss = 0
        kl_loss = 0
        kl_loss_expert = 0
        with tqdm(validloader) as bar:
            with torch.no_grad():
                for i, (xb, yb) in enumerate(bar):
                    xb, yb = xb.to(device), yb.to(device)
                    final_output, kl_loss_gate, kl_loss_expert_usage = net(xb)
                    loss += criterion(final_output, yb) + kl_loss_gate + kl_loss_expert_usage
                    nums += len(yb)
                    kl_loss += kl_loss_gate
                    kl_loss_expert += kl_loss_expert_usage
        valid_loss = loss / nums
        kl_loss_ = kl_loss / nums
        kl_loss_expert_ = kl_loss_expert / nums
        valid_acc = evaluation(net, validloader, device)
        print(f'valid loss: {valid_loss:.4f}, valid accuracy: {valid_acc:.2f}%')
        results.append({'train accuracy': train_acc, 'valid accuracy': valid_acc, 'loss': (losses / nums).sum().item(), 'valid loss': (valid_loss).sum().item()})
        bar.set_description(f'Epoch {epoch + largest}/{epochs}, train accuracy {train_acc:.2f}%, valid accuracy {valid_acc:.2f}%, train loss {losses / nums:.4f}, valid loss {valid_loss:.4f}, kl_loss_gate {kl_loss_:.4f}, kl_loss_expert_usage {kl_loss_expert_:.4f}, learning rate {optimizer.param_groups[0]["lr"]}')
        #save the checkpoint
        scheduler.step()
        # torch.save(net, os.path.join(model_dir, f'cifar10_4x_{epoch + largest}.pth'))
        if valid_acc > best_acc:
            best_acc = valid_acc
            best_idx = epoch + largest
            torch.save(net, os.path.join(model_dir, f'cifar10_4x_best.pth'))
        print(f'best idx: {best_idx}, best accuracy: {best_acc:.2f}%')
        with open(log_path, 'w') as f:
            f.write(json.dumps(results) + '\n')
        wandb.log({
            'epoch': epoch,
            'train_loss': train_loss.item(),
            'valid_loss': valid_loss.item(),
            'train_accuracy': train_acc,
            'valid_accuracy': valid_acc,
            'learning_rate': optimizer.param_groups[0]['lr']
        })
    return train_acc, valid_acc
    ##############################################################################
    #                              END OF YOUR CODE                              #
    ##############################################################################

In [25]:
torch.backends.cudnn.enabled = True

In [ ]:
net.to(device)
def get_last_lr(log_path):
    return 2e-4 # 默认学习率

current_lr = get_last_lr(log_path)
optimizer = torch.optim.Adam(net.parameters(),lr=current_lr,weight_decay=2.5e-4) 
#at the last epoch, lr becomes 0.1 of the starter lr
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 300, eta_min=1e-6)

print(device)
tacc,vacc = train(300, optimizer, scheduler)

cuda


KeyboardInterrupt: 

In [ ]:
from evaluation import evaluation
evaluation(net,validloader,device)

d:\DL\CodingProject2\model_.py:209: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  kl_loss_expert_usage = F.kl_div(F.log_softmax(expert_usage_dist), uniform_dist, reduction='batchmean')


Accuracy of the network on the valid images: 91 %


91.92

## Evaluation

Before submission, please run the following cell to make sure your model can be correctly graded.